# Imports

In [6]:
import numpy as np
import pandas as pd
import socket as sk
import struct as st
import datetime as dt
import ipaddress as ip
from time import perf_counter

# Configurations

In [3]:
# finding the bad guys, dah!

# slowloris 
slowloris_low  = st.unpack('!I', sk.inet_aton('10.128.0.1'))[0]
slowloris_high = st.unpack('!I', sk.inet_aton('10.128.0.50'))[0]

# slowhttptest
slowhttptest_low  = st.unpack('!I', sk.inet_aton('10.128.0.50'))[0]
slowhttptest_high = st.unpack('!I', sk.inet_aton('10.128.0.100'))[0]

# slowloris_ng
slowloris_ng_low  = st.unpack('!I', sk.inet_aton('10.128.0.100'))[0]
slowloris_ng_high = st.unpack('!I', sk.inet_aton('10.128.0.150'))[0]

# defining the TCP flags
tcp_flags = [2, 4, 16, 17, 18, 20, 24, 25, 82, 144, 152, 194]

# Prequel Preprocessing
* this function aims to cast the raw data into a 10 base integer represetation

In [5]:
def prequelProcessing(packages):
    
    dataset = packages.copy()
    
    # Casting IP to a single integer
    dataset['source_ip'] = dataset.source_ip.apply(lambda x: st.unpack('!I', sk.inet_aton(x))[0])
    dataset['dest_ip'] = dataset.dest_ip.apply(lambda x: st.unpack('!I', sk.inet_aton(x))[0])
    
    # Casting Hexa to decimal base
    dataset['tcp_flag'] = dataset.tcp_flag.apply(lambda x: int(x, 16))
    
    # Parsing string to datetime object
    dataset['date'] = dataset['date'] + ' ' + dataset['time']
    dataset['date'] = pd.to_datetime(dataset['date'], format='%Y%m%d %H:%M:%S', utc=True)
    
    # Getting rid of useless columns
    dataset.drop(columns=['data', 'time'], inplace=True)
    
    return dataset

# Features Engineering

In [6]:
def features(grouped_data): 
    
    number_requisitions = np.sum(grouped_data['dest_port'] == 80) + np.sum(grouped_data['dest_port'] == 443)
    number_different_destinations = len(np.unique(grouped_data['dest_ip']))
    mean_frame_length = grouped_data['frame_length'].mean()
    
    data = {
            'number_requisitions'           : [number_requisitions], 
            'number_different_destinations' : [number_different_destinations], 
            'mean_frame_length'             : [mean_frame_length]
           }

    for flag in tcp_flags:
        data['flag_' + str(flag)] = [np.sum(grouped_data['tcp_flag'] == flag)]
    
    return pd.DataFrame(data)

# Normalizing the Data

In [7]:
def turnToPercentil(dataset, summary, column_name):

    for i in range(len(summary.index)):
        
        if summary[column_name][i] > 0:

            data_percentil = dataset.loc[summary.index[i], column_name] / summary[column_name][i]
            dataset.loc[summary.index[i], column_name] = data_percentil.values

In [8]:
def normalizationPerTimePeriod(dataset):
    
    summary = dataset.groupby('date').sum()
    
    column_names= dataset.columns.values
    column_names = np.delete(column_names, 2)
    
    for column in column_names:
        
        turnToPercentil(dataset, summary, column)

# Creating the true label array

In [9]:
def generateLabelColumn(grouped):
    
    # setting all IPs with none intruser type
    grouped['y'] = 0
    
    # resetting the index 
    dataset = grouped.reset_index()
    
    # finding the bad guys
    slowloris    = (dataset.source_ip >= slowloris_low) & (dataset.source_ip < slowloris_high)
    slowhttptest = (dataset.source_ip >= slowhttptest_low) & (dataset.source_ip < slowhttptest_high)
    slowloris_ng = (dataset.source_ip >= slowloris_ng_low) & (dataset.source_ip < slowloris_ng_high)

    # and labeling them
    dataset.loc[slowloris, 'y']    = 1
    dataset.loc[slowhttptest, 'y'] = 1
    dataset.loc[slowloris_ng, 'y'] = 1
    
    # resuming the original index
    dataset.set_index(['date', 'source_ip'], inplace=True)
    
    # getting rid of useless columns
    dataset.drop(columns=['level_2'], inplace=True)
    
    return dataset

# Gathering those functions into a single one

In [10]:
def preprocessing(packages, frequency):
    
    dataset = prequelProcessing(packages)
    
    grouped = dataset.groupby([
            # groupping the data per a specific time frequency
            pd.Grouper(key='date', freq=frequency), 
            # groupping the remaining data by the IPs
            pd.Grouper(key='source_ip')
            # Applying the function who will create the news features
            ]).apply(features)
    
    # normalizing the data
    normalizationPerTimePeriod(grouped)
    
    # generating the true label array
    dataset = generateLabelColumn(grouped)
    
    return dataset

# Testing

In [7]:
packages = pd.read_csv('../dataset/raw/SUEE1.csv', low_memory=False, index_col=[0])
packages.head()

/home/joao/anaconda3/lib/python3.7/site-packages/numpy/lib/arraysetops.py:569: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


,protocol,source_ip,source_port,dest_ip,dest_port,frame_length,tcp_flag,data,date,time
frame,,,,,,,,,,
1,6,192.168.0.1,80,192.168.0.2,39266,66,0x00000011,NaN,1970-01-05,02:41:38
2,6,192.168.0.2,39266,192.168.0.1,80,66,0x00000010,NaN,1970-01-05,02:41:38
3,6,192.168.0.1,80,192.168.0.3,9784,54,0x00000011,NaN,1970-01-05,02:41:38
4,6,192.168.0.3,9784,192.168.0.1,80,54,0x00000010,NaN,1970-01-05,02:41:38
5,6,192.168.0.1,80,192.168.0.4,62170,66,0x00000011,NaN,1970-01-05,02:41:38


In [8]:
packages.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2089436 entries, 1 to 2089436
Data columns (total 10 columns):
protocol        int64
source_ip       object
source_port     int64
dest_ip         object
dest_port       int64
frame_length    int64
tcp_flag        object
data            object
date            object
time            object
dtypes: int64(4), object(6)
memory usage: 175.4+ MB


In [9]:
import sys; sys.path.append('..')

In [10]:
from modules.preprocessing import preprocessing

In [11]:
dataset = preprocessing(packages, '1s')

In [12]:
dataset.head(6)

number_requisitions  \
date                      source_ip                         
1970-01-05 01:38:06+00:00 3232235521             0.607143   
                          3232235529             0.142857   
                          3232235531             0.214286   
                          3232235532             0.000000   
                          3232235547             0.000000   
                          3232235583             0.035714   

                                      number_different_destinations  \
date                      source_ip                                   
1970-01-05 01:38:06+00:00 3232235521                       0.545455   
                          3232235529                       0.090909   
                          3232235531                       0.090909   
                          3232235532                       0.090909   
                          3232235547                       0.090909   
                          3232235583                       0.090909   

                                      mean_frame_length  flag_2  flag_4  \
date                      source_ip                                       
1970-01-05 01:38:06+00:00 3232235521          62.142857    0.50     1.0   
                          3232235529          55.000000    0.25     0.0   
                          3232235531          54.666667    0.25     0.0   
                          3232235532          66.800000    0.00     0.0   
                          3232235547          67.333333    0.00     0.0   
                          3232235583          66.000000    0.00     0.0   

                                      flag_16   flag_17  flag_18  flag_20  \
date                      source_ip                                         
1970-01-05 01:38:06+00:00 3232235521     0.35  0.714286     0.50      0.0   
                          3232235529     0.10  0.000000     0.00      0.0   
                          3232235531     0.15  0.000000     0.00      0.0   
                          3232235532     0.20  0.142857     0.25      0.0   
                          3232235547     0.15  0.142857     0.25      0.0   
                          3232235583     0.05  0.000000     0.00      0.0   

                                       flag_24  flag_25  flag_82  flag_144  \
date                      source_ip                                          
1970-01-05 01:38:06+00:00 3232235521  0.529412      0.0      0.0       0.0   
                          3232235529  0.058824      0.0      0.0       0.0   
                          3232235531  0.117647      0.0      0.0       0.0   
                          3232235532  0.235294      0.0      0.0       0.0   
                          3232235547  0.058824      0.0      0.0       0.0   
                          3232235583  0.000000      0.0      0.0       0.0   

                                      flag_152  flag_194  y  
date                      source_ip                          
1970-01-05 01:38:06+00:00 3232235521       0.0       0.0  0  
                          3232235529       0.0       0.0  0  
                          3232235531       0.0       0.0  0  
                          3232235532       0.0       0.0  0  
                          3232235547       0.0       0.0  0  
                          3232235583       0.0       0.0  0